# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle
from sklearn.utils.multiclass import unique_labels
from skimage.util import random_noise
from sklearn.utils import shuffle
import cv2
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt
%matplotlib inline

# TODO: Fill this in based on where you saved the training and testing data
training_file = "./data/train.p"
validation_file= "./data/valid.p"
testing_file = "./data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

EPOCHS = 20
BATCH_SIZE = 128

# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of validation examples
n_validation = len(X_valid)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(unique_labels(y_train))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
fignum = 0

n_classes = len(unique_labels(y_train))
n_train = len(X_train)
n_average = n_train // n_classes
fignum += 1
plt.figure(fignum)
plt.hist(y_train, bins=n_classes)  # arguments are passed to np.histogram
plt.title("Histogram for training data")
plt.xlabel("Class label")
plt.ylabel("Number of samples")
plt.savefig("output/histogram_training.png")

fignum += 1
plt.figure(fignum)
plt.hist(y_valid, bins=n_classes)  # arguments are passed to np.histogram
plt.title("Histogram for validation data")
plt.xlabel("Class label")
plt.ylabel("Number of samples")
plt.savefig("output/histogram_validation.png")

fignum += 1
plt.figure(fignum)
plt.hist(y_test, bins=n_classes)  # arguments are passed to np.histogram
plt.title("Histogram for test data")
plt.xlabel("Class label")
plt.ylabel("Number of samples")
plt.savefig("output/histogram_test.png")

# Visualizations will be shown in the notebook.

#plt.imshow(cv2.cvtColor(X_train[random.randint(0, n_train)], cv2.COLOR_BGR2RGB))

In [ ]:
sample_image = X_train[34248]
sample_image_gray = cv2.cvtColor(sample_image, cv2.COLOR_RGB2GRAY)

fignum=fignum+1
plt.figure(fignum)
plt.imshow(sample_image)
plt.savefig('output/sample_image.png')

fignum=fignum+1
plt.figure(fignum)
plt.imshow(sample_image_gray, cmap='gray')
plt.savefig('output/sample_image_gray.png')

In [ ]:
n_classes = len(unique_labels(y_train))
n_train = len(X_train)
n_average = n_train // n_classes
hist = []
augment_factor = []
for i in range(n_classes):
    nsamples = sum(y_train == i)
    hist.append(nsamples)
    augment_factor.append(n_average // nsamples)

print(hist)
print(n_average)
print(max(hist))
max_hist = max(hist)
augment_factor = [max_hist // hist[i] for i in range(n_classes)]
print(augment_factor)

In [ ]:
def add_noise(input_image, mean=0, var=10):
    sigma = var ** 0.5
    gaussian = np.random.normal(mean, sigma, input_image.shape)
    noisy_image = np.zeros(input_image.shape, np.float32)
    noisy_image[:, :, :] = input_image[:, :, :] + gaussian
    cv2.normalize(noisy_image, noisy_image, 0, 255, cv2.NORM_MINMAX, dtype=-1)
    noisy_image = noisy_image.astype(np.uint8)
    
    return noisy_image

def expand_dataset(X_input, y_input, new_elements = 1000):
    X_duplicate = []
    y_duplicate = []
    n_output = 0
    for i in range(new_elements):
        i_input = i       
        ilabel = y_input[i_input]
        if augment_factor[ilabel] > 1:
            for _ in range(augment_factor[ilabel]-1):
                n_output += 1
                X_duplicate.append(add_noise(X_input[i_input, :, :, :]))
                y_duplicate.append(y_input[i_input])
        
    return X_duplicate, y_duplicate

X_duplicate, y_duplicate = expand_dataset(X_train, y_train, len(X_train))
X_expanded = np.concatenate((X_train, X_duplicate), axis=0)
y_expanded = np.concatenate((y_train, y_duplicate), axis=0)
X_train = X_expanded
y_train = y_expanded
# print(y_train[-2])
# plt.imshow(X_train[-2])
print(y_duplicate[-2])
plt.imshow(X_duplicate[-2])

In [ ]:
original_image = X_train[34248]
original_image_noise = add_noise(original_image, mean=0, var=10)

fignum=fignum+1
plt.figure(fignum)
plt.imshow(original_image)
plt.savefig('output/original_image.png')

fignum=fignum+1
plt.figure(fignum)
plt.imshow(original_image_noise)
plt.savefig('output/original_image_noise.png')

In [ ]:
n_classes = len(unique_labels(y_train))
n_train = len(X_train)
n_average = n_train // n_classes
fignum += 1
plt.figure(fignum)
plt.hist(y_train, bins=n_classes)  # arguments are passed to np.histogram
plt.title("Histogram for training data after augmentation")
plt.xlabel("Class label")
plt.ylabel("Number of samples")
plt.savefig("output/histogram_training_augment.png")

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.
X_train_gray = np.zeros((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1), dtype=np.float32)
X_valid_gray = np.zeros((X_valid.shape[0], X_valid.shape[1], X_valid.shape[2], 1), dtype=np.float32)
X_test_gray = np.zeros((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1), dtype=np.float32)
for i in range(n_train):
    X_train_gray[i,:,:,0] = cv2.cvtColor(X_train[i], cv2.COLOR_RGB2GRAY)
for i in range(n_validation):
    X_valid_gray[i,:,:,0] = cv2.cvtColor(X_valid[i], cv2.COLOR_RGB2GRAY)
for i in range(n_test):
    X_test_gray[i,:,:,0] = cv2.cvtColor(X_test[i], cv2.COLOR_RGB2GRAY)

#X_train_gray = np.int_(X_train_gray)
#print(X_train_gray.shape)
plt.imshow(np.int_(X_train_gray[2000].squeeze()), cmap='gray')
X_train_gray = (X_train_gray - 128) / 128.0 
X_valid_gray = (X_valid_gray - 128) / 128.0
X_test_gray = (X_test_gray - 128) / 128.0

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
def LeNet(x_input):    
   # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    input1_height = int(x_input.shape[1])
    input1_width = int(x_input.shape[2])
    input1_channels = 1
    
    f1_height = 5
    f1_width = 5
    f1_out_channels = 10
    f1_stride = 1
    f1_pad = 0
    
    output1_height = (input1_height - f1_width + 2*f1_pad) // f1_stride + 1 
    output1_width = (input1_width - f1_width + 2*f1_pad) // f1_stride + 1
    output1_channels = f1_out_channels
    
    #print(output1_height, output1_width, output1_channels)
    
    f1p_height = 2
    f1p_width = 2
    f1p_stride = 2
    f1p_pad = 0
    
    output1p_height = (output1_height - f1p_height + 2*f1p_pad) // f1p_stride + 1 
    output1p_width = (output1_width - f1p_width + 2*f1p_pad) // f1p_stride + 1 
    output1p_channels = output1_channels
    
    #print(output1p_height, output1p_width)
    
    input2_height = output1p_height
    input2_width = output1p_width
    input2_channels = output1p_channels
    
    f2_height = 5
    f2_width = 5
    f2_in_channels = input2_channels
    f2_out_channels = 25
    f2_stride = 1
    f2_pad = 0
    
    output2_height = (input2_height - f2_width + 2*f2_pad) // f2_stride + 1
    output2_width = (input2_width - f2_width + 2*f2_pad) // f2_stride + 1
    output2_channels = f2_out_channels
    
    #print(output2_height, output2_width, output2_channels)
    
    f2p_height = 2
    f2p_width = 2
    f2p_stride = 2
    f2p_pad = 0
    
    output2p_height = (output2_height - f2p_height + 2*f2p_pad) // f2p_stride + 1 
    output2p_width = (output2_width - f2p_width + 2*f2p_pad) // f2p_stride + 1 
    output2p_channels = output2_channels
    
    #print(output2p_height, output2p_width, output2_channels)
    
    input3_length = output2p_height * output2p_width * output2_channels
    
    #print(input3_length, type(input3_length))
    
    input4_length = 120
    input5_length = 84
    
    weights = {
        'wc1': tf.Variable(tf.random_normal([f1_height, f1_width, input1_channels, f1_out_channels], mean=mu, stddev=sigma)),
        'wc2': tf.Variable(tf.random_normal([f2_height, f2_width, f2_in_channels, f2_out_channels], mean=mu, stddev=sigma)),
        'wd1': tf.Variable(tf.random_normal([input3_length, input4_length], mean=mu, stddev=sigma)),
        'wd2': tf.Variable(tf.random_normal([input4_length, input5_length], mean=mu, stddev=sigma)),
        'out': tf.Variable(tf.random_normal([input5_length, n_classes], mean=mu, stddev=sigma))}

    biases = {
        'bc1': tf.Variable(tf.random_normal([f1_out_channels], mean=mu, stddev=sigma)),
        'bc2': tf.Variable(tf.random_normal([f2_out_channels], mean=mu, stddev=sigma)),
        'bd1': tf.Variable(tf.random_normal([input4_length], mean=mu, stddev=sigma)),
        'bd2': tf.Variable(tf.random_normal([input5_length], mean=mu, stddev=sigma)),
        'out': tf.Variable(tf.random_normal([n_classes], mean=mu, stddev=sigma))}
    
    # TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    x = tf.nn.conv2d(x_input, weights['wc1'], strides=[1, f1_stride, f1_stride, 1], padding='VALID')
    x = tf.nn.bias_add(x, biases['bc1'])

    # TODO: Activation.
    x = tf.nn.relu(x)
    #x = tf.nn.dropout(x, keep_prob=0.5)

    # TODO: Pooling. Input = 28x28x10. Output = 14x14x10.
    x = tf.nn.max_pool(x, ksize=[1, f1p_height, f1p_width, 1], strides=[1, f1p_stride, f1p_stride, 1], padding='SAME')

    # TODO: Layer 2: Convolutional. Output = 10x10x25.
    x = tf.nn.conv2d(x, weights['wc2'], strides=[1, f2_stride, f2_stride, 1], padding='VALID')
    x = tf.nn.bias_add(x, biases['bc2'])
    
    # TODO: Activation.
    x = tf.nn.relu(x)
    #x = tf.nn.dropout(x, keep_prob=0.5)

    # TODO: Pooling. Input = 10x10x25. Output = 5x5x25.
    x = tf.nn.max_pool(x, ksize=[1, f2p_height, f2p_width, 1], strides=[1, f2p_stride, f2p_stride, 1], padding='SAME')
    
    # TODO: Flatten. Input = 5x5x25. Output = 625.    
    x = tf.reshape(x, [-1, weights['wd1'].get_shape().as_list()[0]])
    
    # TODO: Layer 3: Fully Connected. Input = 625. Output = 120.
    x = tf.add(tf.matmul(x, weights['wd1']), biases['bd1'])
   
    # TODO: Activation.
    x = tf.nn.relu(x)
    x = tf.nn.dropout(x, keep_prob=0.8)

    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    x = tf.add(tf.matmul(x, weights['wd2']), biases['bd2'])
    
    # TODO: Activation.
    x = tf.nn.relu(x)

    # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
    logits = tf.add(tf.matmul(x, weights['out']), biases['out'])
    
    return logits


### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.


x = tf.placeholder(tf.float32, (None, image_shape[0], image_shape[1], 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)

rate = 0.001

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
predicted_labels = tf.argmax(logits, 1)
expected_labels = tf.argmax(one_hot_y, 1)
#top_labels = tf.nn.top_k(tf.nn.softmax(logits), 5)
top_labels = tf.nn.top_k(logits, 5)
correct_prediction = tf.equal(predicted_labels, expected_labels)
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy, prediction = sess.run([accuracy_operation, predicted_labels], feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return prediction, total_accuracy / num_examples


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train_gray)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train_gray, y_train = shuffle(X_train_gray, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train_gray[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
        
        y_train_pred, train_accuracy = evaluate(X_train_gray, y_train)    
        y_valid_pred, validation_accuracy = evaluate(X_valid_gray, y_valid)
        y_test_pred, test_accuracy = evaluate(X_test_gray, y_test)
        print("EPOCH {} ...".format(i+1))
        print("Train Accuracy = {:.3f}".format(train_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print("Test Accuracy = {:.3f}".format(test_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
## Images taken from 
## https://www.autoeurope.com/travel-guides/germany/germany-road-signs/
## https://en.wikipedia.org/wiki/Road_signs_in_Germany
web_images = ['children_crossing.png',
'german-road-signs-animals.png',
'german-road-signs-traffic.png',  
'germany-speed-limit-sign-60.png',  
'roadwork.png',  
'do_not_enter.png',       
'german-stop-sign.png']
traffic_sign = np.zeros((len(web_images), 32, 32, 1))
for i, img in enumerate(web_images):
    image = cv2.imread(img)
    traffic_sign[i, :, :, 0] = cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), (32, 32))

plt.figure(2, figsize=(32,32))
for featuremap in range(len(traffic_sign)):
    plt.subplot(6,5, featuremap+1) # sets the number of feature maps to show on each row and column
    plt.title(web_images[featuremap]) # displays the feature map number
    plt.imshow(traffic_sign[featuremap,:,:,0], cmap="gray")    


### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.
saver = tf.train.Saver()

number2description = {}
f = open('signnames.csv', 'r')
for line in f:
    number, description = line.split(',')
    number2description[number] =  description
    
with tf.Session() as sess:
    saver.restore(sess, './lenet')
    [traffic_sign_top_labels, traffic_sign_prediction] = sess.run([top_labels, predicted_labels], feed_dict={x: traffic_sign})

sign_descriptions = list(map(lambda x: number2description[str(x)], traffic_sign_prediction))
print('Sign descriptions: {}'.format(sign_descriptions))

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.
values = traffic_sign_top_labels.values
indices = traffic_sign_top_labels.indices
i=0
plt.figure(i)
plt.imshow(traffic_sign[i,:,:,0], cmap='gray')
for j in range(len(indices[i])):
    print(str(values[i][j]), number2description[str(indices[i][j])])

In [ ]:
i=1
plt.figure(i)
plt.imshow(traffic_sign[i,:,:,0], cmap='gray')
for j in range(len(indices[i])):
    print(str(values[i][j]), number2description[str(indices[i][j])])

In [ ]:
i=2
plt.figure(i)
plt.imshow(traffic_sign[i,:,:,0], cmap='gray')
for j in range(len(indices[i])):
    print(str(values[i][j]), number2description[str(indices[i][j])])

In [ ]:
i=3
plt.figure(i)
plt.imshow(traffic_sign[i,:,:,0], cmap='gray')
for j in range(len(indices[i])):
    print(str(values[i][j]), number2description[str(indices[i][j])])

In [ ]:
i=4
plt.figure(i)
plt.imshow(traffic_sign[i,:,:,0], cmap='gray')
for j in range(len(indices[i])):
    print(str(values[i][j]), number2description[str(indices[i][j])])

In [ ]:
i=5
plt.figure(i)
plt.imshow(traffic_sign[i,:,:,0], cmap='gray')
for j in range(len(indices[i])):
    print(str(values[i][j]), number2description[str(indices[i][j])])

In [ ]:
i=6
plt.figure(i)
plt.imshow(traffic_sign[i,:,:,0], cmap='gray')
for j in range(len(indices[i])):
    print(str(values[i][j]), number2description[str(indices[i][j])])

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")